In [13]:
# ---------------- Import Libraries ----------------
import pandas as pd
from datetime import datetime

# ---------------- Load CSV ----------------
file_path = "Week-3-Employee-Data_CLEANED.csv"  # Adjust if needed
try:
    df = pd.read_csv(file_path)
    print("✅ CSV file loaded successfully!\n")
except FileNotFoundError:
    print(f"❌ File not found. Check the path: {file_path}")

# Preview the first 5 rows
df.head()


✅ CSV file loaded successfully!



employee_id first_name last_name                   email gender  \
0         1001        Ken    Naidoo  ken.naidoo@example.com   Male   
1         1002        Sam     Smith      sam.smith@mail.com   Male   
2         1003      Oscar   Mthembu  oscar.mthembu@mail.com   Male   
3         1004        Sam     Brown       sam.brown@example   Male   
4         1005      Faith   Mthembu  faith.mthembu@mail.com   Male   

        department       job_title   hire_date   exit_date is_active  \
0  Human Resources       Sales Rep  2017-05-29         NaN    Active   
1  Human Resources       Sales Rep  2016-04-27         NaN    Active   
2       Operations        Engineer  2015-12-02         NaN  Inactive   
3            Sales  Senior Analyst  2016-01-19         NaN    Active   
4  Human Resources        Engineer  2019-06-18  2022-01-22       NaN   

     salary currency         country         state          city  manager_id  \
0  55286.31      GBP    South Africa  Western Cape           NaN         NaN   
1  29408.75      GBP    South Africa           NaN  Johannesburg      1001.0   
2  35393.84      USD    South Africa           KZN     Cape Town         NaN   
3  39890.20      GBP   United States           KZN           NaN      1003.0   
4  25532.92      GBP  United Kingdom           NaN     Polokwane         NaN   

  performance_score  valid_email  valid_exit_date  
0               NaN         True             True  
1                 C         True             True  
2               NaN         True             True  
3                 C        False             True  
4                 B         True             True

In [14]:
# ---------------- Data Cleaning ----------------
today = pd.to_datetime(datetime.today().date())

# Convert columns to proper types
df['hire_date'] = pd.to_datetime(df['hire_date'], errors='coerce')
df['exit_date'] = pd.to_datetime(df['exit_date'], errors='coerce')
df['salary'] = pd.to_numeric(df['salary'], errors='coerce')

# Standardize is_active column to boolean
df['is_active'] = df['is_active'].str.strip().str.lower().replace({'active': True, 'inactive': False})

# Ensure valid_email is boolean
df['valid_email'] = df['valid_email'].astype(bool)

# Drop rows with missing hire_date
df = df.dropna(subset=['hire_date'])


In [15]:
# ---------------- HR KPI Calculations ----------------

# a) Current headcount
headcount = df[df["is_active"] == True].shape[0]

# b) Hires by Year
df["hire_year"] = df["hire_date"].dt.year
hires_by_year = df["hire_year"].value_counts().sort_index()

# c) Average Salary by Department and Currency
avg_salary = df.groupby(["department", "currency"])["salary"].mean().round(2)

# d) Median Tenure (days) for active employees
df["end_date"] = df["exit_date"].fillna(today)
df["tenure_days"] = (df["end_date"] - df["hire_date"]).dt.days
median_tenure = df[df["is_active"] == True]["tenure_days"].median()

# e) Turnover rate (%)
turnover_rate = (df["exit_date"].notna().sum() / len(df)) * 100


In [16]:
# ---------------- Optional Extras ----------------

# Top 5 managers by team size
top_managers = df.groupby("manager_id")["employee_id"].count().sort_values(ascending=False).head(5)

# Percent of invalid emails
invalid_email_pct = (df["valid_email"].eq(False).sum() / len(df)) * 100


In [17]:
summary_data = {
    "KPI": [
        "Current Headcount",
        "Median Tenure (days)",
        "Turnover Rate (%)",
        "Percent of Invalid Emails"
    ],
    "Value": [
        headcount,
        median_tenure,
        round(turnover_rate, 2),
        round(invalid_email_pct, 2)
    ]
}

summary_df = pd.DataFrame(summary_data)

In [18]:

# Display KPI Table
print("📊 HR KPI Summary")
display(summary_df)

# Display Hires by Year
print("\n📈 Hires by Year")
display(hires_by_year.to_frame(name="Number of Hires"))

# Display Average Salary
print("\n💰 Average Salary by Department & Currency")
display(avg_salary.to_frame(name="Average Salary"))

# Display Top Managers
print("\n👨‍💼 Top 5 Managers by Team Size")
display(top_managers.to_frame(name="Team Size"))

📊 HR KPI Summary


KPI    Value
0          Current Headcount    52.00
1       Median Tenure (days)  2444.50
2          Turnover Rate (%)    31.33
3  Percent of Invalid Emails    31.33


📈 Hires by Year


Number of Hires
hire_year                 
2015                    18
2016                    18
2017                    27
2018                    20
2019                    18
2020                    13
2021                    13
2022                    23


💰 Average Salary by Department & Currency


Average Salary
department             currency                
Engineering            GBP             44174.25
                       USD             29981.22
                       ZAR             33195.09
Finance                GBP             31096.12
                       USD             87676.22
                       ZAR             37262.52
Human Resources        GBP             36016.42
                       USD             54174.53
                       ZAR             58245.97
Information Technology GBP             35309.93
                       USD             36616.06
                       ZAR             35202.39
Operations             GBP             28019.71
                       USD             34355.64
                       ZAR             33195.55
Sales                  GBP             25249.97
                       USD             35982.63
                       ZAR             32720.59


👨‍💼 Top 5 Managers by Team Size


Team Size
manager_id           
999999.0           12
1006.0              7
1003.0              5
1001.0              4
1016.0              4